In [2]:
from __future__ import print_function
%matplotlib inline
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

Using Theano backend.


# Note types
- 0: nothing
- 1: step
- 2: hold start
- 3: hold/roll end
- 4: roll start
- M: mine

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [3]:
samples_back_included = 8
num_classes = 7
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, notes, index):
    if index < 0:
        return [0] * num_features
    return beat_features[index]

def get_class_for_index(notes, index):
    if index < 0:
        return [1, 0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    steps += (holds + rolls)
    return [int(i) for i in [steps == 0 and mines == 0, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key, is_full):
    X = []
    y = []
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            row_y = get_class_for_index(notes, i)
            if is_full or (not (row_y == 0 and random.randint(0, 5) != 0)):
                features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, notes, i - j)]
                features.append(i % 48)
                features.append(get_beat_importance(i))
                features.append(i / 48)
                features.append(num_notes - i / 48)
                X.append(features)
                y.append(row_y)
    return np.array(X), np.array(y)

# Total 243 songs
def build_training_data(songs_start, songs_end, is_full = False):
    X = []
    y = []
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use[songs_start:songs_end]:
        song_X, song_y = get_features_for_song(song_data[0], is_full)
        X.extend(song_X)
        y.extend(song_y)
    return X, y

In [4]:
# start with each input secion maps to one note
# train model for that (just list comprhension on noets for contains 1 maps to true)

# then move to bars eg section of 4 bars maps to output for each note
# error = probability of note being true vs was it really

# try bar + prev notes (home use weird dimensioned data?) to predict next notes

# try feeding in non structured data (bpm, position of time in song, song length, 
# things about feel of song (generated features))
# try using keras merge layer to add extra features

In [5]:
X_train, y_train = build_training_data(0, 243, True)

In [9]:
class_weight = {
    0: 1,
    1: 2,
    2: 4,
    3: 8,
    4: 4,
    5: 4,
    6: 6
}

In [10]:
beat_feature_model = Sequential()

beat_feature_model.add(Dense(500, input_dim=num_features_total, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(500, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(500, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(num_classes, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('softmax'))

beat_feature_model.compile(loss='categorical_crossentropy',
                           optimizer='adadelta',
                           metrics=['accuracy'])

In [11]:
beat_feature_model.fit(np.array(X_train), np.array(y_train), nb_epoch=5, batch_size=5) #, class_weight=class_weight)
beat_feature_model.save('models/beat_importance_model_classes_full_testing.h5')

Epoch 1/5
601265/601265 [==============================] - 29176s - loss: 0.5004 - acc: 0.8506 
Epoch 2/5
601265/601265 [==============================] - 2191s - loss: 0.4688 - acc: 0.8557  
Epoch 3/5
601265/601265 [==============================] - 2007s - loss: 0.4661 - acc: 0.8564  
Epoch 4/5
601265/601265 [==============================] - 2012s - loss: 0.4650 - acc: 0.8569  
Epoch 5/5
601265/601265 [==============================] - 2014s - loss: 0.4647 - acc: 0.8570  


In [12]:
beat_feature_model = Sequential()
# 5186, 8482
beat_feature_model.add(Dense(500, input_dim=num_features_total, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(500, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(500, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(num_classes, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('softmax'))

beat_feature_model.compile(loss='categorical_crossentropy',
                           optimizer='adadelta',
                           metrics=['accuracy'])

In [13]:
beat_feature_model.fit(np.array(X_train), np.array(y_train), nb_epoch=5, batch_size=5, class_weight=class_weight)
beat_feature_model.save('models/beat_importance_model_classes_full_testing_with_classes.h5')

Epoch 1/5
601265/601265 [==============================] - 2046s - loss: 0.9665 - acc: 0.8285  
Epoch 2/5
601265/601265 [==============================] - 1932s - loss: 0.9158 - acc: 0.8232  
Epoch 3/5
601265/601265 [==============================] - 1961s - loss: 0.9128 - acc: 0.8250  
Epoch 4/5
601265/601265 [==============================] - 1894s - loss: 0.9109 - acc: 0.8270  
Epoch 5/5
601265/601265 [==============================] - 1940s - loss: 0.9086 - acc: 0.8281  


In [14]:
def step_song(key, clf):
    song_X, song_y = get_features_for_song(key, True)
    new_song_y = clf.predict(song_X)
    
    pd.DataFrame(new_song_y).to_csv('generated_data/{0}_note_classes_generated.csv'.format(key), index=False)

In [15]:
beat_feature_model = load_model('models/beat_importance_model_classes_full_testing.h5')

In [16]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
failed_song_keys = []
for song_data in songs_to_use:
    try:
        step_song(song_data[0], beat_feature_model)
    except:
        print ('\nError loading song')
        print (song_data[0])
        failed_song_keys.append(song_data[0])


Error loading song
In The Groove~I Think I Like That Sound

Error loading song
In The Groove~Remember December

Error loading song
In The Groove~Torn

Error loading song
In The Groove~Walking on Fire

Error loading song
In The Groove 2~!

Error loading song
In The Groove 2~Fleadh Uncut

Error loading song
In The Groove 2~Hardcore Symphony

Error loading song
In The Groove 2~Holy Guacamole

Error loading song
In The Groove 2~Reactor

Error loading song
In The Groove 3~DJ Superstar

Error loading song
In The Groove 3~Land of Imagination

Error loading song
In The Groove 3~Online

Error loading song
In The Groove 3~Partyman

Error loading song
In The Groove Rebirth~Space Space Shooter

Error loading song
In The Groove Rebirth +~Roppongi Carillon

Error loading song
In The Groove Rebirth 2 (BETA)~Death From Above

Error loading song
In The Groove Rebirth 2 (BETA)~Elder God Shrine

Error loading song
In The Groove Rebirth 2 (BETA)~La Samba de la Vida

Error loading song
In The Groove Rebir

In [43]:
failed_song_keys = ['In The Groove~I Think I Like That Sound',
 'In The Groove~Remember December',
 'In The Groove~Torn',
 'In The Groove~Walking on Fire',
 'In The Groove 2~!',
 'In The Groove 2~Fleadh Uncut',
 'In The Groove 2~Hardcore Symphony',
 'In The Groove 2~Holy Guacamole',
 'In The Groove 2~Reactor',
 'In The Groove 3~DJ Superstar',
 'In The Groove 3~Land of Imagination',
 'In The Groove 3~Online',
 'In The Groove 3~Partyman',
 'In The Groove Rebirth~Space Space Shooter',
 'In The Groove Rebirth +~Roppongi Carillon',
 'In The Groove Rebirth 2 (BETA)~Death From Above',
 'In The Groove Rebirth 2 (BETA)~Elder God Shrine',
 'In The Groove Rebirth 2 (BETA)~La Samba de la Vida',
 'In The Groove Rebirth 2 (BETA)~Night Flight to Tokyo',
 'In The Groove Rebirth 2 (BETA)~Ninja Boy',
 'In The Groove Rebirth 2 (BETA)~Style on my Speed Dial',
 'In The Groove Rebirth 2 (BETA)~Team Rocket',
 'In The Groove Rebirth 2 (BETA)~Tike Tike Kardi',
 'In The Groove Rebirth 2 (BETA)~Toy Soldiers',
 'Piece of Cake 2~(-[Jayce]-) - Divine Olivine - [10]',
 'Piece of Cake 5~(-[Jayce]-) - I Run NY - [09]']